In [2]:
%%capture
!pip install --upgrade langchain langgraph faiss-cpu transformers sentence-transformers textblob pandas scikit-learn langchain-community openai

In [4]:
import pandas as pd
import numpy as np
from langchain.agents import tool
from langchain_core.runnables import RunnableLambda
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFacePipeline
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langgraph.graph import END, StateGraph
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [5]:
from google.colab import files

# This will prompt a file upload dialog
uploaded = files.upload()

# Print the uploaded filename(s)
print("\nUploaded files:")
for filename in uploaded.keys():
    print(f" - {filename}")

Saving Bitext_Sample_Customer_Service_Testing_Dataset.csv to Bitext_Sample_Customer_Service_Testing_Dataset.csv

Uploaded files:
 - Bitext_Sample_Customer_Service_Testing_Dataset.csv


In [6]:
import pandas as pd

# Read the uploaded CSV file
df = pd.read_csv('Bitext_Sample_Customer_Service_Testing_Dataset.csv')

In [8]:
import pandas as pd

# Load dataset
df = pd.read_csv('Bitext_Sample_Customer_Service_Testing_Dataset.csv')

# Create clean_text column safely
df['clean_text'] = df['utterance'].str.lower().str.replace(r'[^\w\s]', '', regex=True)

# Verify columns
print("Dataset columns:", df.columns.tolist())
print("First 5 clean_text values:")
print(df['clean_text'].head())

Dataset columns: ['utterance', 'intent', 'category', 'tags', 'clean_text']
First 5 clean_text values:
0         i have a question about cancelling an order
1                help canceling the order i have made
2    i do not know how to cancel an order i have made
3            where can i cancel the last order i made
4                   i dont want the last order i made
Name: clean_text, dtype: object


In [9]:
# Train intent classifier
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['clean_text'])
y = df['intent']
intent_classifier = LinearSVC()
intent_classifier.fit(X, y)

LinearSVC()

In [10]:
# 4. Create knowledge base for RAG
docs = [f"Intent: {row['intent']}\nCategory: {row['category']}\nUtterance: {row['utterance']}"
        for _, row in df.iterrows()]
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
knowledge_base = FAISS.from_texts(docs, embeddings)

<ipython-input-10-92432fa400a5>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.wa

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

In [13]:
# 5. Load local LLM
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)  # Correct class

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=150,
    temperature=0.7
)
llm = HuggingFacePipeline(pipeline=pipe)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
<ipython-input-13-80aee014039b>:13: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [94]:
# 6. Define proper state structure
from typing import TypedDict, Annotated, Union
from langgraph.graph import END, StateGraph

In [95]:
class AgentState(TypedDict):
    user_input: str
    intent: str
    sentiment: str
    rag_response: Union[str, None]
    enriched_context: str
    next_agent: str
    draft_response: str
    final_response: str
    escalation_status: str
    feedback_prompt: str

In [97]:
# --- Build Workflow Graph ---
workflow = StateGraph(AgentState)

# Add nodes with proper state handling
def intent_classifier_node(state: AgentState):
    return {"intent": classify_intent(state["user_input"])}

def sentiment_analyzer_node(state: AgentState):
    return {"sentiment": analyze_sentiment(state["user_input"])}

def rag_agent_node(state: AgentState):
    query = state["user_input"]
    docs = knowledge_base.similarity_search(query, k=3)
    context = "\n".join([d.page_content for d in docs])
    prompt = PromptTemplate.from_template(
        "Answer using only the context:\n{context}\n\nQuestion: {query}\nAnswer:"
    )
    chain = prompt | llm
    return {"rag_response": chain.invoke({"context": context, "query": query})}

def context_enricher_node(state: AgentState):
    enriched = f"User Input: {state['user_input']}\nDetected Intent: {state['intent']}\nSentiment: {state['sentiment']}"
    if state.get("rag_response"):
        enriched += f"\nRelevant Knowledge: {state['rag_response']}"
    return {"enriched_context": enriched}

# FIXED QUERY ROUTER - ENSURES VALID AGENT NAME
def query_router_node(state: AgentState):
    intent = state["intent"]
    print(f"Routing intent: {intent}")  # Debug print

# Define agent mapping
    agent_map = {
        'cancel_order': 'order_agent',
        'change_order': 'order_agent',
        'place_order': 'order_agent',
        'track_order': 'order_agent',
        'create_account': 'account_agent',
        'delete_account': 'account_agent',
        'edit_account': 'account_agent',
        'switch_account': 'account_agent',
        'shipping': 'shipping_agent',
        'delivery': 'shipping_agent',
        'payment': 'payment_agent',
        'refund': 'payment_agent'
    }

# Find matching agent
    for key, agent in agent_map.items():
        if key in intent.lower():
            return {"next_agent": agent}

    # Default to general_agent
    return {"next_agent": "general_agent"}

def order_agent_node(state: AgentState):
    prompt = PromptTemplate.from_template(
        "You are an Order Specialist. Respond to this customer query:\n{user_input}\n\nContext:\n{enriched_context}\n\nResponse:"
    )
    chain = prompt | llm
    return {"draft_response": chain.invoke(state)}

def account_agent_node(state: AgentState):
    prompt = PromptTemplate.from_template(
        "You are an Account Specialist. Respond to this customer query:\n{user_input}\n\nContext:\n{enriched_context}\n\nResponse:"
    )
    chain = prompt | llm
    return {"draft_response": chain.invoke(state)}

def shipping_agent_node(state: AgentState):
    prompt = PromptTemplate.from_template(
        "You are a Shipping Specialist. Respond to this customer query:\n{user_input}\n\nContext:\n{enriched_context}\n\nResponse:"
    )
    chain = prompt | llm
    return {"draft_response": chain.invoke(state)}

def payment_agent_node(state: AgentState):
    prompt = PromptTemplate.from_template(
        "You are a Payment Specialist. Respond to this customer query:\n{user_input}\n\nContext:\n{enriched_context}\n\nResponse:"
    )
    chain = prompt | llm
    return {"draft_response": chain.invoke(state)}

def response_generator_node(state: AgentState):
    prompt = PromptTemplate.from_template(
        "Refine this draft response considering the user's sentiment ({sentiment}) and intent ({intent}):\n\nDraft: {draft_response}\n\nPolished response:"
    )
    chain = prompt | llm
    return {"final_response": chain.invoke(state)}

def escalation_detector_node(state: AgentState):
    if state["sentiment"] == "negative":
        return {"escalation_status": "escalation_needed"}
    return {"escalation_status": "no_escalation"}

def feedback_collector_node(state: AgentState):
    return {"feedback_prompt": "Was this solution helpful? Please rate 1-5:"}


In [98]:
# Add nodes
workflow.add_node("intent_classifier", intent_classifier_node)
workflow.add_node("sentiment_analyzer", sentiment_analyzer_node)
workflow.add_node("rag_agent", rag_agent_node)
workflow.add_node("context_enricher", context_enricher_node)
workflow.add_node("query_router", query_router_node)
workflow.add_node("order_agent", order_agent_node)
workflow.add_node("account_agent", account_agent_node)
workflow.add_node("shipping_agent", shipping_agent_node)
workflow.add_node("payment_agent", payment_agent_node)
workflow.add_node("general_agent", order_agent_node)
workflow.add_node("response_generator", response_generator_node)
workflow.add_node("escalation_detector", escalation_detector_node)
workflow.add_node("feedback_collector", feedback_collector_node)


In [99]:
# Set entry point
workflow.set_entry_point("intent_classifier")

# Add edges
workflow.add_edge("intent_classifier", "sentiment_analyzer")
workflow.add_edge("sentiment_analyzer", "rag_agent")
workflow.add_edge("rag_agent", "context_enricher")
workflow.add_edge("context_enricher", "query_router")

# FIXED ROUTER BRANCHES - HANDLE UNEXPECTED VALUES
workflow.add_conditional_edges(
    "query_router",
    lambda state: state.get("next_agent", "general_agent"),
    {
        "order_agent": "order_agent",
        "account_agent": "account_agent",
        "shipping_agent": "shipping_agent",
        "payment_agent": "payment_agent",
        "general_agent": "general_agent"
    }
)

In [100]:
# Connect to response generator
workflow.add_edge("order_agent", "response_generator")
workflow.add_edge("account_agent", "response_generator")
workflow.add_edge("shipping_agent", "response_generator")
workflow.add_edge("payment_agent", "response_generator")
workflow.add_edge("general_agent", "response_generator")

# Escalation check
workflow.add_edge("response_generator", "escalation_detector")
workflow.add_conditional_edges(
    "escalation_detector",
    lambda state: state.get("escalation_status", "no_escalation"),
    {
        "escalation_needed": "feedback_collector",
        "no_escalation": "feedback_collector"
    }
)

# Final step
workflow.add_edge("feedback_collector", END)

# Compile graph
app = workflow.compile()

In [101]:
# --- Run the workflow ---
def run_workflow(query):
    inputs = {"user_input": query}
    try:
        response = app.invoke(inputs)
        return {
            "final_response": response.get("final_response", "No response generated"),
            "escalation": response.get("escalation_status", "no_escalation"),
            "feedback_prompt": response.get("feedback_prompt", "")
        }
    except Exception as e:
        print(f"Error in workflow: {e}")
        return {
            "final_response": "Sorry, I encountered an error processing your request.",
            "escalation": "escalation_needed",
            "feedback_prompt": ""
        }

In [102]:
# Test with a single query first
test_query = "I need to cancel my recent order immediately!"
print(f"\n{'='*50}")
print(f"TESTING WITH QUERY: {test_query}")
result = run_workflow(test_query)
print("\nFINAL RESPONSE:", result["final_response"])
print("ESCALATION NEEDED:", result["escalation"] == "escalation_needed")
print("FEEDBACK PROMPT:", result["feedback_prompt"])
print(f"{'='*50}\n")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



TESTING WITH QUERY: I need to cancel my recent order immediately!


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Routing intent: cancel_order


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



FINAL RESPONSE: I need to cancel my recent order immediately!
ESCALATION NEEDED: False
FEEDBACK PROMPT: Was this solution helpful? Please rate 1-5:



In [103]:
# Fixing the Order Agent Prompt:

# Current problematic prompt:
"You are an Order Specialist. Respond to this customer query:\n{user_input}\n\nContext:\n{enriched_context}\n\nResponse:"

# Improved version:
prompt = PromptTemplate.from_template(
    "As an Order Specialist, help the customer with their request: {user_input}\n\n"
    "Context:\n{enriched_context}\n\n"
    "Your response should directly address their request to cancel an order:"
)

In [104]:
#Improving the Response Generator:

# Current prompt:
"Refine this draft response considering the user's sentiment ({sentiment}) and intent ({intent}):\n\nDraft: {draft_response}\n\nPolished response:"

# Improved version:
prompt = PromptTemplate.from_template(
    "You are a customer service response editor. Refine this draft response to be helpful, professional, "
    "and directly address the user's intent to {intent}. Consider their {sentiment} sentiment:\n\n"
    "Draft Response: {draft_response}\n\n"
    "Improved Final Response:"
)

In [105]:
# Removing temperature parameter from pipeline
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=150
    # temperature=0.7  # Remove this line
)

Device set to use cpu


In [110]:
#  DEPLOYMENT

import time  # to import time at the top

def run_workflow(query):
    inputs = {"user_input": query}
    try:
        start_time = time.time()
        state = app.invoke(inputs)
        processing_time = time.time() - start_time
        return {
            "intent": state.get("intent", "N/A"),
            "sentiment": state.get("sentiment", "N/A"),
            "next_agent": state.get("next_agent", "N/A"),
            "final_response": state.get("final_response", "I'm sorry, I couldn't process that request."),
            "escalation": state.get("escalation_status", "no_escalation"),
            "feedback_prompt": state.get("feedback_prompt", "Was this helpful?"),
            "processing_time": f"{processing_time:.2f} seconds"
        }
    except Exception as e:
        print(f"Error in workflow: {e}")
        return {
            "final_response": "I'm experiencing technical difficulties. Please try again later.",
            "escalation": "escalation_needed",
            "feedback_prompt": "",
            "processing_time": "N/A"
        }

def interactive_demo():
    """Interactive chat interface with all enhancements"""
    print("="*80)
    print("AI CUSTOMER SERVICE AGENT")
    print("="*80)
    print("\nWelcome to our AI Customer Service! I can help with:")
    print(" • Order cancellations and changes")
    print(" • Shipping and delivery questions")
    print(" • Payment and refund issues")
    print(" • Account management")
    print(" • Product information")
    print("\nType 'exit' at any time to end the conversation")
    print("="*80)
    print("\n[System Initialized] Assistant is ready to help!\n")

    conversation_history = []
    exchange_count = 0
    escalation_count = 0

    while True:
        # Get user input with error handling
        user_input = input("\nCustomer: ").strip()

        # Exit condition
        if user_input.lower() in ['exit', 'quit', 'bye']:
            break

        # Handle empty input
        if not user_input:
            print("\nAssistant: I didn't receive your message. Could you please repeat?")
            conversation_history.append("Customer: [EMPTY INPUT]")
            conversation_history.append("Assistant: I didn't receive your message. Could you please repeat?")
            continue

        # Show typing indicator
        print("\nAssistant is thinking...", end='', flush=True)
        start_time = time.time()  # Start timing

        # Run workflow
        result = run_workflow(user_input)

        # Calculate processing time
        processing_time = time.time() - start_time

        # Clear typing indicator
        print("\r" + " "*30 + "\r", end='')

        # Update conversation stats
        exchange_count += 1
        if result['escalation'] == "escalation_needed":
            escalation_count += 1

        # Store conversation
        conversation_history.append(f"Customer: {user_input}")
        conversation_history.append(f"Assistant: {result['final_response']}")

        # Display response
        print(f"Assistant: {result['final_response']}")

        # Show escalation if needed
        if result['escalation'] == "escalation_needed":
            print("\n[System] ⚠️ Your issue has been escalated to senior support!")

        # Collect feedback
        print(f"\n[System] {result['feedback_prompt']}")
        feedback = input("Your rating (1-5): ").strip()

        # Handle feedback
        if feedback.isdigit() and 1 <= int(feedback) <= 5:
            print("[System] Thank you for your feedback!")
            conversation_history.append(f"Feedback: {feedback}/5")
        else:
            print("[System] Invalid rating. Feedback skipped.")
            conversation_history.append("Feedback: Not provided")

        # Show processing time
        print(f"[System] Processed in {result['processing_time']}")

        # Add separation
        print("\n" + "-"*60)

    # Session summary
    print("\n" + "="*80)
    print("SESSION SUMMARY")
    print("="*80)
    print(f"Total exchanges: {exchange_count}")
    print(f"Escalations: {escalation_count}")
    print(f"Average rating: {'N/A' if exchange_count == 0 else 'Calculated from logs'}")
    print("\nThank you for using our AI customer service!")

    # Save conversation log
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    filename = f"conversation_log_{timestamp}.txt"
    with open(filename, "w") as f:
        f.write("\n".join(conversation_history))
    print(f"\nConversation log saved to '{filename}'")

# Start the interactive demo
interactive_demo()

AI CUSTOMER SERVICE AGENT

Welcome to our AI Customer Service! I can help with:
 • Order cancellations and changes
 • Shipping and delivery questions
 • Payment and refund issues
 • Account management
 • Product information

Type 'exit' at any time to end the conversation

[System Initialized] Assistant is ready to help!


Customer: I want to cancel my order

Assistant is thinking...

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Routing intent: cancel_order


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Assistant: I don't know how to cancel an order

[System] Was this solution helpful? Please rate 1-5:
Your rating (1-5): 1
[System] Thank you for your feedback!
[System] Processed in 4.80 seconds

------------------------------------------------------------

Customer: change an order

Assistant is thinking...

...

Assistant: yes

[System] Was this solution helpful? Please rate 1-5:
Your rating (1-5): 2
[System] Thank you for your feedback!
[System] Processed in 3.39 seconds

------------------------------------------------------------

Customer: I need assistance to remove an item from an order

Assistant is thinking...

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Routing intent: change_order


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Assistant: Is this what you're looking for?

[System] Was this solution helpful? Please rate 1-5:
Your rating (1-5): 2
[System] Thank you for your feedback!
[System] Processed in 3.83 seconds

------------------------------------------------------------

Customer: exit

SESSION SUMMARY
Total exchanges: 3
Escalations: 0
Average rating: Calculated from logs

Thank you for using our AI customer service!

Conversation log saved to 'conversation_log_20250528-200944.txt'
